In [47]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler


# GridSearchCV

In [ ]:
# Load dataset
path = '../Heart_disease_cleveland_new.csv'
df = pd.read_csv(path)  

# Preprocess
X = df.drop("target", axis=1).values
y = df["target"].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
def build_model(hidden_layers=1, neurons=(32,), activation='relu', optimizer='adam', learning_rate=0.01):
    model = Sequential()
    model.add(Dense(neurons[0], activation=activation, input_shape=(X.shape[1],)))

    if hidden_layers == 2 and len(neurons) > 1:
        model.add(Dense(neurons[1], activation=activation))

    model.add(Dense(1, activation='sigmoid'))

    if optimizer == 'SGD':
        opt = SGD(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [37]:
from scikeras.wrappers import KerasClassifier

clf = KerasClassifier(
    model=build_model,
    hidden_layers=1,
    neurons=(32,),
    activation='relu',
    optimizer='adam',
    learning_rate=0.01,
    epochs=10,
    batch_size=32,
    verbose=0
)

In [38]:
param_grid = {
    'hidden_layers': [1, 2],
    'neurons': [(32,), (64, 32)],
    'activation': ['relu'],
    'optimizer': ['adam', 'SGD'],
    'learning_rate': [0.01, 0.001],
    'batch_size': [32],
    'epochs': [10]
}

In [39]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3)
grid.fit(X_train, y_train)

print("Best score:", grid.best_score_)
print("Best params:", grid.best_params_)

Best score: 0.8180555555555555
Best params: {'activation': 'relu', 'batch_size': 32, 'epochs': 10, 'hidden_layers': 2, 'learning_rate': 0.01, 'neurons': (32,), 'optimizer': 'SGD'}


# RandomizedSearchCV

In [40]:
from sklearn.model_selection import RandomizedSearchCV

In [41]:
param_distributions = {
    'hidden_layers': [1, 2],
    'neurons': [(32,), (64, 32), (16,), (32, 16)],
    'activation': ['relu'],
    'optimizer': ['adam', 'SGD'],
    'learning_rate': [0.01, 0.001, 0.005],
    'batch_size': [16, 32],
    'epochs': [50]
}

In [45]:
from scikeras.wrappers import KerasClassifier

clf = KerasClassifier(
    model=build_model,
    hidden_layers=1,
    neurons=(32,),
    activation='relu',
    optimizer='adam',
    learning_rate=0.01,
    epochs=50,
    batch_size=32,
    verbose=0
)


In [46]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'hidden_layers': [1, 2],
    'neurons': [(32,), (64, 32), (16,), (32, 16)],
    'activation': ['relu'],
    'optimizer': ['adam', 'SGD'],
    'learning_rate': [0.01, 0.001, 0.005],
    'batch_size': [16, 32],
    'epochs': [50]
}

random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_distributions,
    n_iter=10,  # Try 10 random combinations
    cv=3,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Best params:", random_search.best_params_)
print("Best score:", random_search.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best params: {'optimizer': 'adam', 'neurons': (32,), 'learning_rate': 0.01, 'hidden_layers': 1, 'epochs': 50, 'batch_size': 32, 'activation': 'relu'}
Best score: 0.8099279835390947
